# HDF5とPyTablesによる巨大な異種データ混合テーブルの操作
データ型の混じったテーブルを格納：操作することも可能。

## 準備
PyTablesがインストールされていること。

## 手順

### NumPyとPyTablesをインポート

In [1]:
import numpy as np
import tables as tb

### HDF5ファイルの作成

In [2]:
f = tb.open_file('myfile.h5', 'w') # 書き込み権限でh5ファイルを開く

### ファイルに格納するH5テーブルを作成

In [4]:
dtype = np.dtype([('city', 'S64'), ('population', 'i4')]) # npのdtype関数に、2つのタプルデータの入った、1つのリストを格納

### ルートに作成テーブルを格納

In [5]:
table = f.create_table('/', 'table1', dtype) # 階層'/'に、テーブル名'table1'として、h5テーブルdtypeを格納

### 既存テーブルにデータを追加

In [6]:
table.append([('Brussels', 1138854), ('London', 8308369), ('Paris', 2243833)]) # テーブル"table1'に3つのデータを追加

### テーブルの内容変更をコミット

In [7]:
table.flush() # 行追加による変更をコミット

### テーブルへのアクセス

#### テーブル全体にアクセス

In [8]:
table[:] # テーブル全体にアクセス

array([(b'Brussels', 1138854), (b'London', 8308369), (b'Paris', 2243833)], 
      dtype=[('city', 'S64'), ('population', '<i4')])

#### 特定列にアクセス

In [9]:
table.col('city') # 特定col(=列)等にもアクセス可能

array([b'Brussels', b'London', b'Paris'], 
      dtype='|S64')

#### 行条件による検索

In [11]:
[row['city'] for row in table.where('population>2e6')] # 行に条件を付加して、該当データを取り出す

[b'London', b'Paris']

#### 特定行にアクセス

In [12]:
table[1] # 特定の行にアクセス

(b'London', 8308369)

## 解説
　テーブルは最初から作ることも、既存のNumPy配列やpandasのDataFrameからも構築できる。NumPyなら、列名は辞書を使ってNumPyのdtypeとして指定。pandasなら、NumPy配列やDaataFrameから引き継ぐ。
 
　table.append()によるテーブルへのデータ行追加は効率的。データを1行追加するには、まずrow = table.rowにより新しいデータ行のインスタンスを作成する。辞書として行データを設定した後、row.append()を実行するとテーブルの最後に新しい行が追加される。flush()の実行で、変更がファイルに書き込まれる。